In [2]:
import numpy as np
import pandaset as ps
import transforms3d

In [7]:
dataset = ps.DataSet('/zfs/penshorn/master_thesis/datasets/raw/PandaSet')

sequences = dataset.sequences()

seq = dataset['011']
seq.load_camera()

seq.camera['front_camera'].poses[0]


{'position': {'x': 0.05324751932344672,
  'y': 0.5418887113856523,
  'z': 1.816017925733849},
 'heading': {'w': 0.7100100192413354,
  'x': -0.7040353885222189,
  'y': 0.00945366271558075,
  'z': -0.011426834493699793}}

In [4]:
def coordinates_conversion(pose:np.ndarray):
        c2w = pose.copy()
        flip_mat = np.array([
            [1, 0, 0, 0],
            [0, -1, 0, 0],
            [0, 0, -1, 0],
            [0, 0, 0, 1]
        ])
        flip_mat2 = np.array([
            [0, -1, 0, 0],
            [1, 0, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
        # just for debug purpose, uncomment this part if used to train with nerfstudio
        c2w[0:3,2] *= -1 # flip the y and z axis
        c2w[0:3,1] *= -1
        c2w = c2w[[1,0,2,3],:]
        c2w[2,:] *= -1 # flip whole world upside down
        return(flip_mat2 @ flip_mat @ c2w)


def convert_to_nerfstudio_camera_orientation(self, pose:np.ndarray):
    # swap x and y and invert old y axis
    flip_mat2 = np.array([
        [-1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1]
    ])
    return flip_mat2 @ pose

def pose_to_Tr(pose):
        T = np.array([pose['position']['x'],pose['position']['y'],pose['position']['z']])
        R = transforms3d.quaternions.quat2mat(np.array([
            pose['heading']['w'],
            pose['heading']['x'],
            pose['heading']['y'],
            pose['heading']['z'],

        ]))
        Tr = np.eye(4)
        Tr[:3,:3] = R
        Tr[:3,3] = T
        return Tr

def rotation_matrix_x(angle):
    return np.array([
        [1, 0, 0],
        [0, np.cos(angle), -np.sin(angle)],
        [0, np.sin(angle), np.cos(angle)]
    ])

def rot_mat_to_transform(rot_mat):
    transform = np.eye(4)
    transform[:3,:3] = rot_mat
    return transform

In [6]:
cam_orientation = np.eye(4)

cam_orientation[:3,3] = np.array([1, 2, 3])

# rotation matrix around the x axis
cam_rotation = rotation_matrix_x(np.pi)

# Explicitly defined to prevent small floating point values != 0
rot_x_pi = np.array([
    [1, 0, 0],
    [0, -1, 0],
    [0, 0, -1]
])

print(cam_orientation)
print(cam_rotation)
# print(cam_rotation @ cam_orientation)
print(np.sin(np.pi))
print(np.cos(np.pi/2))

print(coordinates_conversion(cam_orientation))

print(rot_mat_to_transform(rot_x_pi) @ cam_orientation)



[[1. 0. 0. 1.]
 [0. 1. 0. 2.]
 [0. 0. 1. 3.]
 [0. 0. 0. 1.]]
[[ 1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00 -1.0000000e+00 -1.2246468e-16]
 [ 0.0000000e+00  1.2246468e-16 -1.0000000e+00]]
1.2246467991473532e-16
6.123233995736766e-17
[[ 1.  0.  0.  1.]
 [ 0. -1.  0.  2.]
 [ 0.  0. -1.  3.]
 [ 0.  0.  0.  1.]]
[[ 1.  0.  0.  1.]
 [ 0. -1.  0. -2.]
 [ 0.  0. -1. -3.]
 [ 0.  0.  0.  1.]]
